# **<span style="color:lightgreen;">Scaling, Imputing and Heatmaps with Supervised Learning</span>**

## **Purpose of Notebook**


- Load datasets
- Sample to balance size
- Dummy fill NaNs
- Encode categoricals
- Scale numericals
- Supervised imputation
- Correlation heatmap
- Feature selection (top 10)

---

**Load needed Libraries**

In [14]:
# Basic Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

# Preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer

# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Evaluation
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, mean_squared_error
)

# Clustering (if needed)
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score



**Load in the cleaned datasets**

In [15]:
# Display all columns of a pandas DataFrame when printed
pd.set_option('display.max_columns', None)

# US Dataset
US_data = pd.read_csv('/Users/mahnooriqbal/COMP702 Project/ML-AI-Risk-Analysis-AV-Data-/Datasets/US-cleaned_data.csv')


# UK Dataset
UK_data = pd.read_csv('/Users/mahnooriqbal/COMP702 Project/ML-AI-Risk-Analysis-AV-Data-/Datasets/UK-cleaned_data.csv')

print("US Dataset Shape:", US_data.shape)
print("UK Dataset Shape:", UK_data.shape)

US Dataset Shape: (4372, 21)
UK Dataset Shape: (232365, 21)


In [16]:
# Identify numerical and categorical columns
numerical_columns = ['Posted Speed Limit (MPH)']
categorical_columns = [
    'Make', 'Model', 'Model Year', 'ADS Equipped?',
    'Automation System Engaged?', 'City', 'State', 'Roadway Type', 'Roadway Surface',
    'Lighting', 'Crash With', 'Highest Injury Severity Alleged',
    'SV Pre-Crash Movement', 'SV Contact Area', 'Weather', 'Country'
]
datetime_columns = ['Incident Date', 'Incident Time (24:00)']
indexing_columns = ['Report ID', 'Report Version']

**Fill in Missing Data**



In [17]:
from prettytable import PrettyTable

# Function to calculate and display missing values
def display_missing_values(dataset, dataset_name):
    features_with_na = [feature for feature in dataset.columns if dataset[feature].isnull().sum() > 0]
    
    # Create a PrettyTable
    table = PrettyTable()
    table.field_names = ["Feature", "Percentage of Missing Values"]
    
    for feature in features_with_na:
        missing_percentage = np.round(dataset[feature].isnull().mean() * 100, 2)
        table.add_row([feature, f"{missing_percentage} %"])
    
    print(f"Missing Values in {dataset_name}:")
    print(table)

# Check missing values for both datasets
display_missing_values(UK_data, "UK Dataset")
display_missing_values(US_data, "US Dataset")


Missing Values in UK Dataset:
+---------+------------------------------+
| Feature | Percentage of Missing Values |
+---------+------------------------------+
|  Model  |            1.11 %            |
+---------+------------------------------+
Missing Values in US Dataset:
+--------------------------+------------------------------+
|         Feature          | Percentage of Missing Values |
+--------------------------+------------------------------+
|        Model Year        |            0.32 %            |
|      Incident Date       |            0.43 %            |
|  Incident Time (24:00)   |            4.6 %             |
| Posted Speed Limit (MPH) |           11.34 %            |
+--------------------------+------------------------------+


# **Encode Catergorical and Time Series Data**

In [18]:
# STEP 1: Make a copy
data_copy = US_data.copy()

# STEP 2: Drop index and datetime columns
data_copy.drop(columns=indexing_columns, inplace=True)
data_copy.drop(columns=datetime_columns, inplace=True)

# STEP 3: Encode all categorical columns
label_encoders = {}
for col in categorical_columns:
    if col in data_copy.columns:
        le = LabelEncoder()
        data_copy[col] = le.fit_transform(data_copy[col].astype(str))
        label_encoders[col] = le

# STEP 4: Combine date and time into a single datetime column
US_data['incident_datetime'] = pd.to_datetime(
    US_data['Incident Date'].astype(str) + ' ' + US_data['Incident Time (24:00)'],
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)
# STEP 5: Extract datetime features
data_copy['incident_year'] = US_data['incident_datetime'].dt.year
data_copy['incident_month'] = US_data['incident_datetime'].dt.month
data_copy['incident_day'] = US_data['incident_datetime'].dt.day
data_copy['incident_hour'] = US_data['incident_datetime'].dt.hour
data_copy['incident_minute'] = US_data['incident_datetime'].dt.minute
data_copy['incident_dayofweek'] = US_data['incident_datetime'].dt.dayofweek


**Scale Values**

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_copy_scaled = data_copy.copy()
data_copy_scaled[data_copy.columns] = scaler.fit_transform(data_copy)
print(data_copy_scaled.head())

# Check missing values for both datasets
display_missing_values(UK_data, "UK Dataset")
display_missing_values(US_data, "US Dataset")

       Make     Model  Model Year  ADS Equipped?  Automation System Engaged?  \
0  0.397059  0.493151    0.615385            0.0                         1.0   
1  0.397059  0.493151    0.615385            1.0                         1.0   
2  0.397059  0.502283    0.538462            1.0                         1.0   
3  0.397059  0.493151    0.615385            0.0                         0.0   
4  0.397059  0.502283    0.615385            0.0                         1.0   

       City     State  Roadway Type  Roadway Surface  \
0  0.518072  0.725490      0.571429              0.0   
1  0.518781  0.725490      0.571429              0.0   
2  0.907867  0.039216      0.000000              0.0   
3  0.555634  0.372549      0.000000              0.0   
4  0.593905  0.901961      0.000000              0.0   

   Posted Speed Limit (MPH)  Lighting  Crash With  \
0                    0.3125  0.166667    0.533333   
1                    0.3125  0.166667    0.533333   
2                    0.

# **Imputing Methods**
- mean/median/mode
- k Means
- random 
- regression model
- k nearest neighbours
- last occured carried forward

### **Mean, Median, Mode**
- Best for: Small amounts of missing numerical data.
- How it works: Replaces missing values with the mean (for normally distributed data), median (for skewed data), or mode (for categorical variables).
- Limitations: Can distort distributions and underestimate variance.

In [20]:
def mode_mean_imputation(data, numerical_columns, categorical_columns, datetime_columns):
    """
    Impute missing values in numerical and categorical columns using mean for numerical and mode for categorical and datetime.
    
    Parameters:
    - data: DataFrame with missing values
    - numerical_columns: List of numerical columns to impute
    - categorical_columns: List of categorical columns to impute
    - datetime_columns: List of datetime columns to impute
    
    Returns:
    - DataFrame with imputed values
    """
    # Impute numerical columns with mean
    for col in numerical_columns:
        if col in data.columns:
            data[col].fillna(data[col].mean(), inplace=True)
    
    # Impute categorical columns with mode
    for col in categorical_columns:
        if col in data.columns:
            data[col].fillna(data[col].mode()[0], inplace=True)
    
    # Impute datetime columns with mode
    for col in datetime_columns:
        if col in data.columns:
            data[col].fillna(data[col].mode()[0], inplace=True)
    
    return data

mode_imputed_df = mode_mean_imputation(US_data.copy(), numerical_columns, categorical_columns, datetime_columns)
from tabulate import tabulate

print(tabulate(mode_imputed_df.head(), headers='keys', tablefmt='psql'))





+----+-------------+------------------+---------------+---------+--------------+-----------------+------------------------------+-----------------+-------------------------+--------------------+---------+-------------------+-------------------+----------------------------+--------------------+--------------------+-----------------------------------+-------------------------+-----------+--------------------------------+-----------+---------------------+
|    | Report ID   |   Report Version | Make          | Model   |   Model Year | ADS Equipped?   | Automation System Engaged?   | Incident Date   | Incident Time (24:00)   | City               | State   | Roadway Type      | Roadway Surface   |   Posted Speed Limit (MPH) | Lighting           | Crash With         | Highest Injury Severity Alleged   | SV Pre-Crash Movement   | Weather   | SV Contact Area                | Country   | incident_datetime   |
|----+-------------+------------------+---------------+---------+--------------+------

/var/folders/0k/whv99z5n5rnb1xp8n07krpcw0000gn/T/ipykernel_8791/2025267401.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mean(), inplace=True)
/var/folders/0k/whv99z5n5rnb1xp8n07krpcw0000gn/T/ipykernel_8791/2025267401.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

### **Random Imputation**
- **Best for:** Maintaining original distribution with enough data  
- **How it works:** Replaces missing by random sampling from existing values  
- **Limitations:** Adds randomness; may create unrealistic combinations  


In [21]:
import numpy as np

def random_imputation(data, numerical_columns, categorical_columns, datetime_columns):
    """
    Impute missing values by randomly sampling from non-missing values in each column.
    
    Parameters:
    - data: DataFrame with missing values
    - numerical_columns: List of numerical columns to impute
    - categorical_columns: List of categorical columns to impute
    - datetime_columns: List of datetime columns to impute
    
    Returns:
    - DataFrame with imputed values
    """
    for col in numerical_columns + categorical_columns + datetime_columns:
        if col in data.columns:
            non_null_values = data[col].dropna().values
            # Only impute if there are missing values and non-null values exist
            if data[col].isnull().sum() > 0 and len(non_null_values) > 0:
                # Randomly choose from non-null values to fill missing
                random_samples = np.random.choice(non_null_values, size=data[col].isnull().sum(), replace=True)
                data.loc[data[col].isnull(), col] = random_samples
                
    return data

# Use your data and columns as before
mode_imputed_df = random_imputation(US_data.copy(), numerical_columns, categorical_columns, datetime_columns)

from tabulate import tabulate
print(tabulate(mode_imputed_df.head(), headers='keys', tablefmt='psql'))


+----+-------------+------------------+---------------+---------+--------------+-----------------+------------------------------+-----------------+-------------------------+--------------------+---------+-------------------+-------------------+----------------------------+--------------------+--------------------+-----------------------------------+-------------------------+-----------+--------------------------------+-----------+---------------------+
|    | Report ID   |   Report Version | Make          | Model   |   Model Year | ADS Equipped?   | Automation System Engaged?   | Incident Date   | Incident Time (24:00)   | City               | State   | Roadway Type      | Roadway Surface   |   Posted Speed Limit (MPH) | Lighting           | Crash With         | Highest Injury Severity Alleged   | SV Pre-Crash Movement   | Weather   | SV Contact Area                | Country   | incident_datetime   |
|----+-------------+------------------+---------------+---------+--------------+------

### **Last Occurance Imputation**
- **Best for:** Maintaining original distribution with enough data  
- **How it works:** Replaces missing by random sampling from existing values  
- **Limitations:** Adds randomness; may create unrealistic combinations  


In [22]:
def last_occurrence_imputation(data, numerical_columns, categorical_columns, datetime_columns):
    """
    Impute missing values using backward fill (last occurrence) for numerical, categorical, and datetime columns.
    
    Parameters:
    - data: DataFrame with missing values
    - numerical_columns: List of numerical columns to impute
    - categorical_columns: List of categorical columns to impute
    - datetime_columns: List of datetime columns to impute
    
    Returns:
    - DataFrame with imputed values
    """
    for col in numerical_columns + categorical_columns + datetime_columns:
        if col in data.columns:
            data[col].fillna(method='bfill', inplace=True)
            
    return data

# Usage example
mode_imputed_df = last_occurrence_imputation(US_data, numerical_columns, categorical_columns, datetime_columns)

from tabulate import tabulate
print(tabulate(mode_imputed_df.head(), headers='keys', tablefmt='psql'))


+----+-------------+------------------+---------------+---------+--------------+-----------------+------------------------------+-----------------+-------------------------+--------------------+---------+-------------------+-------------------+----------------------------+--------------------+--------------------+-----------------------------------+-------------------------+-----------+--------------------------------+-----------+---------------------+
|    | Report ID   |   Report Version | Make          | Model   |   Model Year | ADS Equipped?   | Automation System Engaged?   | Incident Date   | Incident Time (24:00)   | City               | State   | Roadway Type      | Roadway Surface   |   Posted Speed Limit (MPH) | Lighting           | Crash With         | Highest Injury Severity Alleged   | SV Pre-Crash Movement   | Weather   | SV Contact Area                | Country   | incident_datetime   |
|----+-------------+------------------+---------------+---------+--------------+------

/var/folders/0k/whv99z5n5rnb1xp8n07krpcw0000gn/T/ipykernel_8791/643073262.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(method='bfill', inplace=True)
/var/folders/0k/whv99z5n5rnb1xp8n07krpcw0000gn/T/ipykernel_8791/643073262.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data[col].fillna(method='bfill', inplace=True)


### Regression Imputer
- **Best for:** When strong relationships exist between features  
- **How it works:** Predicts missing values using regression models on other features  
- **Limitations:** Assumes linearity; risk of overfitting if data is small  

In [23]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
import pandas as pd
from tabulate import tabulate

regression_df = US_data.copy()
# Impute only numerical columns
imputer = IterativeImputer(random_state=0)
imputed_array = imputer.fit_transform(regression_df[numerical_columns])
imputed_numerical_df = pd.DataFrame(imputed_array, columns=numerical_columns)

# For categorical: fill with mode or use another strategy
imputed_categorical_df = regression_df[categorical_columns].fillna(regression_df[categorical_columns].mode().iloc[0])

# For datetime: forward fill as an example
imputed_datetime_df = regression_df[datetime_columns].fillna(method='ffill')
imputed_df = pd.concat([imputed_numerical_df, imputed_categorical_df, imputed_datetime_df], axis=1)
print(tabulate(imputed_df.head(), headers='keys', tablefmt='psql'))


+----+----------------------------+---------------+---------+--------------+-----------------+------------------------------+--------------------+---------+-------------------+-------------------+--------------------+--------------------+-----------------------------------+-------------------------+--------------------------------+-----------+-----------+-----------------+-------------------------+
|    |   Posted Speed Limit (MPH) | Make          | Model   |   Model Year | ADS Equipped?   | Automation System Engaged?   | City               | State   | Roadway Type      | Roadway Surface   | Lighting           | Crash With         | Highest Injury Severity Alleged   | SV Pre-Crash Movement   | SV Contact Area                | Weather   | Country   | Incident Date   | Incident Time (24:00)   |
|----+----------------------------+---------------+---------+--------------+-----------------+------------------------------+--------------------+---------+-------------------+-------------------+

/var/folders/0k/whv99z5n5rnb1xp8n07krpcw0000gn/T/ipykernel_8791/1622105651.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  imputed_datetime_df = regression_df[datetime_columns].fillna(method='ffill')


### **Comparison of Imputer Qualities**

**Create a controlled test set with known missing values**
- To fairly compare imputers, you need to:
- Select a subset of your complete dataset with no missing values.
- Introduce missing values artificially (e.g., 10% randomly) so you know the ground truth.

**Numerical Columns**

In [24]:

# 1. Drop rows with missing values to get clean baseline
complete_data = data_copy_scaled.dropna()
numerical_data = complete_data[numerical_columns]

# 2. Randomly mask 20% of numerical values
data_masked = numerical_data.copy()
mask = np.random.rand(*data_masked.shape) < 0.2
data_masked = data_masked.mask(mask)

# 3. Imputation Methods
# Mean
mean_imp = SimpleImputer(strategy='mean')
mean_result = pd.DataFrame(mean_imp.fit_transform(data_masked), columns=numerical_columns)

# Mode
mode_imp = SimpleImputer(strategy='most_frequent')
mode_result = pd.DataFrame(mode_imp.fit_transform(data_masked), columns=numerical_columns)

# Regression
reg_imp = IterativeImputer(random_state=0)
reg_result = pd.DataFrame(reg_imp.fit_transform(data_masked), columns=numerical_columns)

# Last Occurrence (Forward Fill)
last_occurrence_result = data_masked.fillna(method='ffill')

# Random Imputation
random_result = data_masked.copy()
for col in numerical_columns:
    missing_idx = random_result[col].isna()
    if missing_idx.any():
        random_sample = random_result[col].dropna().sample(missing_idx.sum(), replace=True, random_state=0).values
        random_result.loc[missing_idx, col] = random_sample

# 4. RMSE calculation at only the masked (missing) positions
def masked_rmse(imputed, original, mask):
    rmse_total = 0
    n = 0
    for i, col in enumerate(numerical_columns):
        col_mask = mask[:, i]
        if np.any(col_mask):
            mse = mean_squared_error(original[col_mask, i], imputed[col_mask, i])
            rmse_total += np.sqrt(mse)
            n += 1
    return rmse_total / n

# 5. Convert to arrays
original_array = numerical_data.values
mean_array = mean_result.values
mode_array = mode_result.values
reg_array = reg_result.values
last_occ_array = last_occurrence_result.values
random_array = random_result.values

# 6. Compute RMSE for each method
rmse_scores = {
    "Mean": masked_rmse(mean_array, original_array, mask),
    "Mode": masked_rmse(mode_array, original_array, mask),
    "Regression": masked_rmse(reg_array, original_array, mask),
    "Last Occurrence": masked_rmse(last_occ_array, original_array, mask),
    "Random": masked_rmse(random_array, original_array, mask)
}

# 7. Print results
table_data = [[method, f"{score:.4f}"] for method, score in rmse_scores.items()]
print(tabulate(table_data, headers=["Imputation Method", "RMSE Score"], tablefmt="psql"))



+---------------------+--------------+
| Imputation Method   |   RMSE Score |
|---------------------+--------------|
| Mean                |       0.2266 |
| Mode                |       0.3261 |
| Regression          |       0.2266 |
| Last Occurrence     |       0.2173 |
| Random              |       0.318  |
+---------------------+--------------+


/var/folders/0k/whv99z5n5rnb1xp8n07krpcw0000gn/T/ipykernel_8791/263511584.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  last_occurrence_result = data_masked.fillna(method='ffill')


**Catergorical Columns**

In [25]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from tabulate import tabulate

US_data = US_data.copy()
# ----------------------------------------------
# 1. Drop rows with missing values in categorical columns to get clean baseline
complete_data = US_data.dropna(subset=categorical_columns)

categorical_data = complete_data[categorical_columns]

# ----------------------------------------------
# 2. Randomly mask 20% of categorical data
data_masked = categorical_data.copy()
mask = np.random.rand(*data_masked.shape) < 0.2
data_masked = data_masked.mask(mask)

# ----------------------------------------------
# 3. Imputation Methods

# Mode imputation
mode_imp = SimpleImputer(strategy='most_frequent')
mode_result = pd.DataFrame(mode_imp.fit_transform(data_masked), columns=categorical_columns)

# Forward fill imputation (with backward fill to handle NaNs at top)
last_occurrence_result = data_masked.ffill().bfill()

# Random imputation
random_result = data_masked.copy()
for col in categorical_columns:
    missing_idx = random_result[col].isna()
    if missing_idx.any():
        random_sample = random_result[col].dropna().sample(missing_idx.sum(), replace=True, random_state=0).values
        random_result.loc[missing_idx, col] = random_sample

# ----------------------------------------------
# 4. Encode categorical columns as integers for accuracy calculation
def encode_df(df):
    encoded_df = pd.DataFrame()
    encoders = {}
    for col in categorical_columns:
        encoded_df[col], encoders[col] = pd.factorize(df[col])
    return encoded_df, encoders

original_encoded, encoders = encode_df(categorical_data)
mode_encoded, _ = encode_df(mode_result)
last_occ_encoded, _ = encode_df(last_occurrence_result)
random_encoded, _ = encode_df(random_result)

# ----------------------------------------------
# 5. Calculate accuracy at masked positions
def masked_accuracy(imputed, original, mask):
    total_acc = 0
    n = 0
    for i, col in enumerate(categorical_columns):
        col_mask = mask[:, i]
        if np.any(col_mask):
            acc = accuracy_score(original[col_mask, i], imputed[col_mask, i])
            total_acc += acc
            n += 1
    return total_acc / n

# Convert mask DataFrame to numpy array if needed
mask_array = mask.values if isinstance(mask, pd.DataFrame) else mask

# ----------------------------------------------
# 6. Calculate accuracy scores for each imputation method
accuracy_scores = {
    "Mode": masked_accuracy(mode_encoded.values, original_encoded.values, mask_array),
    "Last Occurrence": masked_accuracy(last_occ_encoded.values, original_encoded.values, mask_array),
    "Random": masked_accuracy(random_encoded.values, original_encoded.values, mask_array)
}

# ----------------------------------------------
# 7. Print results in table format
table_data = [[method, f"{score:.4f}"] for method, score in accuracy_scores.items()]
print(tabulate(table_data, headers=["Imputation Method", "Accuracy Score"], tablefmt="psql"))



+---------------------+------------------+
| Imputation Method   |   Accuracy Score |
|---------------------+------------------|
| Mode                |           0.3594 |
| Last Occurrence     |           0.3973 |
| Random              |           0.2812 |
+---------------------+------------------+


**Date Time columns**

In [28]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from tabulate import tabulate

US_data = US_data.copy()

# ----------------------------------------------
# 1. Drop rows with missing values in datetime columns to get clean baseline
complete_data = US_data.dropna(subset=datetime_columns)

# Select categorical columns from the cleaned data
categorical_data = complete_data[categorical_columns]

# ----------------------------------------------
# 2. Randomly mask 20% of categorical data
data_masked = categorical_data.copy()
mask = np.random.rand(*data_masked.shape) < 0.2
data_masked = data_masked.mask(mask)

# ----------------------------------------------
# 3. Imputation Methods

# Mode imputation
mode_imp = SimpleImputer(strategy='most_frequent')
mode_result = pd.DataFrame(mode_imp.fit_transform(data_masked), columns=categorical_columns)

# Forward fill imputation (with backward fill to handle NaNs at top)
last_occurrence_result = data_masked.ffill().bfill()

# Random imputation
random_result = data_masked.copy()
for col in categorical_columns:
    missing_idx = random_result[col].isna()
    if missing_idx.any():
        random_sample = random_result[col].dropna().sample(missing_idx.sum(), replace=True, random_state=0).values
        random_result.loc[missing_idx, col] = random_sample

# ----------------------------------------------
# 4. Encode categorical columns as integers for accuracy calculation
def encode_df(df):
    encoded_df = pd.DataFrame()
    encoders = {}
    for col in categorical_columns:
        encoded_df[col], encoders[col] = pd.factorize(df[col])
    return encoded_df, encoders

original_encoded, encoders = encode_df(categorical_data)
mode_encoded, _ = encode_df(mode_result)
last_occ_encoded, _ = encode_df(last_occurrence_result)
random_encoded, _ = encode_df(random_result)

# ----------------------------------------------
# 5. Calculate accuracy at masked positions
def masked_accuracy(imputed, original, mask):
    total_acc = 0
    n = 0
    for i, col in enumerate(categorical_columns):
        col_mask = mask[:, i]
        if np.any(col_mask):
            acc = accuracy_score(original[col_mask, i], imputed[col_mask, i])
            total_acc += acc
            n += 1
    return total_acc / n

# Convert mask DataFrame to numpy array if needed
mask_array = mask.values if isinstance(mask, pd.DataFrame) else mask

# ----------------------------------------------
# 6. Calculate accuracy scores for each imputation method
accuracy_scores = {
    "Mode": masked_accuracy(mode_encoded.values, original_encoded.values, mask_array),
    "Last Occurrence": masked_accuracy(last_occ_encoded.values, original_encoded.values, mask_array),
    "Random": masked_accuracy(random_encoded.values, original_encoded.values, mask_array)
}

# ----------------------------------------------
# 7. Print results in table format
table_data = [[method, f"{score:.4f}"] for method, score in accuracy_scores.items()]
print(tabulate(table_data, headers=["Imputation Method", "Accuracy Score"], tablefmt="psql"))


+---------------------+------------------+
| Imputation Method   |   Accuracy Score |
|---------------------+------------------|
| Mode                |           0.3688 |
| Last Occurrence     |           0.4285 |
| Random              |           0.2828 |
+---------------------+------------------+


### **Final Imputations**

For US 
- Numerical = Last Occurance
- Catergorical = Last Occurance
- Datetime = Last Occurance

## **Sample Datasets**

In [26]:
# Downsample the large dataset
UK_data = UK_data.sample(n=4000, random_state=42)
UK_data.shape

(4000, 21)

5. Imbalanced Datasets: knn for minority

## **Outliers**

**DBSCAN**

In [ ]:
from sklearn.impute import SimpleImputer

# If data_array is a NumPy array:
imputer = SimpleImputer(strategy='most_frequent')
data_imputed = imputer.fit_transform(data_array)
from sklearn.cluster import DBSCAN

# Set DBSCAN parameters
dbscan = DBSCAN(eps=0.5, min_samples=5, metric='hamming')  # Use 'hamming' for categorical
labels = dbscan.fit_predict(data_imputed)
from sklearn.metrics import silhouette_score
import numpy as np

# Only calculate score if more than 1 cluster (excluding -1 for noise)
if len(set(labels)) > 1 and len(set(labels)) != 2 or -1 not in labels:
    score = silhouette_score(data_imputed, labels, metric='hamming')
    print("Silhouette Score:", score)
else:
    print("Too few clusters to compute silhouette score.")
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
data_2d = pca.fit_transform(data_imputed)

plt.figure(figsize=(8, 6))
scatter = plt.scatter(data_2d[:, 0], data_2d[:, 1], c=labels, cmap='tab10', s=50)
plt.title("DBSCAN Clustering on Encoded Categorical Data")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.colorbar(scatter, label='Cluster')
plt.show()

## **Create Heatmap**

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# STEP 5: Now you can compute correlation safely
correlations = data_copy.corr()['Highest Injury Severity Alleged'].abs().dropna().sort_values(ascending=False)


# Plot heatmap with correlation values
plt.figure(figsize=(10, 6))
sns.heatmap(data_copy.corr(), annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)
plt.title("Feature Correlation Heatmap with Values")
plt.show()


# STEP 6: Top 10 most correlated (excluding the target itself)
most_correlated_features = correlations.iloc[1:11]
print("Top 10 most correlated features with 'Highest Severity Injury Alleged':")
print(most_correlated_features)

# Filter dataset to top features
data_filtered = data_copy[most_correlated_features.index]
print("Filtered dataset shape:", data_filtered.shape)
